In [2]:
#%%
import cv2
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from PIL import Image


# Loading all relevant datafiles:

dictionary (as pickle - deserializing), csv, meta (json)

pckl: {0 : (['masks', 'x', 'y', 'z', 'time', 'area'])}

In [146]:
Exp_2_pckl={}
path = 'C:/Users/srboval1/OneDrive - Aalto University/Instru/Datafiles/Exp2/'
with open(path + 'ExpDesig2ID_.pkl', 'rb') as f:
    Exp_2_pckl = pickle.load(f)

print(Exp_2_pckl.keys())
print(f"Content of pickle: {Exp_2_pckl[0].keys()}")
print()

Exp_2_csv = pd.read_csv(path +'/ExpDesign2_.csv', index_col=None) #length same as number of keys in pckl; group by measurement_id and well_id -> to group also the identical spheroids in pckl
print(f"Content of csv: {Exp_2_csv.columns}")
print()

Exp_2_json = pd.read_json('C:/Users/srboval1/InstruProject/NikonPipes/dataStore/metalib.json') 
print(f"Content of json: {Exp_2_json['240520'].keys()}")


dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

## Keys of nested dictionaries


scripts for understanding the structure

In [128]:
print('Pckl: Number of individual spheroids (- each will have multiple timestamps as distinct masks):',len(Exp_2_pckl.keys()))
print(f"Csv length:{(Exp_2_csv.shape)}")
print(f"Pickle number of main keys: {len(Exp_2_pckl.keys())}")
print(f"Csv columns: {Exp_2_csv.columns.to_list()}")
print("")
print("Checking 'm' pixel size keys-dates from json:")
print(Exp_2_json.get(key="240521").get("m"))

def search_json(parameter):
    pixel_transf = Exp_2_json.get(key=str(parameter)).get("m")
    return pixel_transf

print(search_json(240521))


Pckl: Number of individual spheroids (- each will have multiple timestamps as distinct masks): 408
Csv length:(408, 10)
Pickle number of main keys: 408
Csv columns: ['Unnamed: 0', 'seeding_density', 'cell_label', 'well_id', 'measurement_id', 'matrix', 'ID', 'day', 'ID_running', 'location']

Checking 'm' pixel size keys-dates from json:
0.660045336571487
0.660045336571487


Cleaning, ordering columns, grouping

In [188]:
# Exp_2_csv = Exp_2_csv.drop(columns=(['Unnamed: 0']))

order_csv = ['ID_running','day', 'matrix', 'cell_label','seeding_density','measurement_id','well_id']  
Exp_2_csv = Exp_2_csv[order_csv]

# print(Exp_2_csv.head())
# print('Number of groups in csv after measurement_id and well_id grouping:',(Exp_2_csv.groupby(['measurement_id', 'well_id'])).ngroups)

# #avoid pandas conflict of copy/view
# Exp_2_csv= Exp_2_csv.copy()

# #setting id for each distinct spheroid - grouped by all parameters that must match for the same spheroid; shall be same as above
# Exp_2_csv['spheroid_id'] = Exp_2_csv.groupby(['measurement_id', 'well_id']).ngroup()

# Exp_2_csv['spheroid_id'] = Exp_2_csv.groupby(['measurement_id', 'well_id']).ngroup()

# print(for i in Exp_2_csv['cell_label'].unique)
print(Exp_2_csv['matrix'].unique())

for i, row in enumerate(Exp_2_csv[Exp_2_csv['matrix'] == 'IPNmix'].iterrows()):
    print(row)



['IPN3' 'IPN22' 'coll' '22mM' 'collagen' 'IPN3mM' 'IPNmix' 'IPN22mM'
 '2mgml']
(176, ID_running            176
day                241025
matrix             IPNmix
cell_label         MCF10A
seeding_density       500
measurement_id          0
well_id                 0
Name: 176, dtype: object)
(177, ID_running            177
day                241025
matrix             IPNmix
cell_label         MCF10A
seeding_density       500
measurement_id          0
well_id                 1
Name: 177, dtype: object)
(178, ID_running            178
day                241025
matrix             IPNmix
cell_label         MCF10A
seeding_density       500
measurement_id          0
well_id                 3
Name: 178, dtype: object)
(179, ID_running            179
day                241025
matrix             IPNmix
cell_label         MCF10A
seeding_density       500
measurement_id          0
well_id                 4
Name: 179, dtype: object)
(180, ID_running            180
day                241025
matrix 

In [131]:

concentration_mapping = {
'2mgml': '1',
'3mM': '2',
'40mM': '3'
}
material_mapping = {
'2mgml': '1',
'3mM': '2',
'40mM': '3'
}
reverse_material_mapping = {
'2mgml':'collagen 2mg/ml',
'3mM':'IPN 2.5mM',
'40mM':'IPN 40mM'
}
cell_mapping = {
'MCF10A': '1',
'DCIS.COM': '2',
'MCF10A.DCIS.COM': '2',
'MCF10ADCIS.com': '2',
'MCF10AT': '3'
}


In [132]:
# print(Exp_2_csv['matrix'].unique())
# print(Exp_2_csv['cell_label'].unique())

a = Exp_2_csv[Exp_2_csv['matrix']=='IPNmix']

print(a)

     ID_running     day  matrix cell_label  seeding_density  measurement_id  \
176         176  241025  IPNmix     MCF10A              500               0   
177         177  241025  IPNmix     MCF10A              500               0   
178         178  241025  IPNmix     MCF10A              500               0   
179         179  241025  IPNmix     MCF10A              500               0   
180         180  241025  IPNmix     MCF10A              500               0   
181         181  241025  IPNmix    MCF10AT              500               0   
182         182  241025  IPNmix    MCF10AT              500               0   
183         183  241025  IPNmix    MCF10AT              500               0   
184         184  241025  IPNmix    MCF10AT              500               0   
185         185  241025  IPNmix    MCF10AT              500               0   
186         186  241026  IPNmix     MCF10A              500               0   
187         187  241026  IPNmix     MCF10A          

Checking pixel size converstions to match

In [133]:
pixel_size = 0.32*1e-6

print('List of contour coordinates:',Exp_2_pckl[2]["masks"][7])
print('Linda calculated area [pixels]:',cv2.contourArea(Exp_2_pckl[2]["masks"][10]))
print('re-calculated area in [m]:',cv2.contourArea(Exp_2_pckl[2]["masks"][10])*(0.660535171355723**2))

print('Arttu calculated area:',Exp_2_pckl[2]["area"][10])

List of contour coordinates: [[[1046 1166]]

 [[1045 1167]]

 [[1044 1167]]

 [[1043 1168]]

 [[1042 1168]]

 [[1041 1169]]

 [[1040 1170]]

 [[1039 1170]]

 [[1038 1170]]

 [[1037 1170]]

 [[1036 1171]]

 [[1035 1171]]

 [[1034 1171]]

 [[1033 1171]]

 [[1032 1172]]

 [[1031 1172]]

 [[1030 1173]]

 [[1029 1173]]

 [[1028 1174]]

 [[1027 1174]]

 [[1026 1174]]

 [[1025 1175]]

 [[1024 1175]]

 [[1023 1175]]

 [[1022 1176]]

 [[1021 1177]]

 [[1020 1178]]

 [[1019 1178]]

 [[1018 1178]]

 [[1017 1179]]

 [[1016 1180]]

 [[1015 1180]]

 [[1014 1181]]

 [[1013 1182]]

 [[1012 1182]]

 [[1011 1182]]

 [[1010 1183]]

 [[1009 1184]]

 [[1009 1185]]

 [[1008 1186]]

 [[1007 1186]]

 [[1006 1186]]

 [[1005 1187]]

 [[1004 1187]]

 [[1003 1187]]

 [[1002 1188]]

 [[1001 1188]]

 [[1000 1188]]

 [[ 999 1188]]

 [[ 998 1188]]

 [[ 997 1189]]

 [[ 996 1189]]

 [[ 995 1189]]

 [[ 994 1189]]

 [[ 993 1190]]

 [[ 992 1190]]

 [[ 991 1190]]

 [[ 990 1191]]

 [[ 989 1191]]

 [[ 988 1192]]

 [[ 987 119

In [149]:
parameter= 240521
Exp_2_json.get(key=str(parameter)).get("m")

json_days= Exp_2_json.keys()
csv_days = Exp_2_csv["day"].unique()
print(len(json_days))

 
print('No of unique days in metafile:',(Exp_2_json['241022']))
print('Unique days in metafile:',(Exp_2_json.keys()))


print('No of unique days in csv:',len(Exp_2_csv["day"].unique()))
print('Unique days in csv:',(Exp_2_csv["day"].unique()))


53
No of unique days in metafile: coords             [[[971, 727], [1527, 1511]], [[814, 648], [142...
matrix                                                        IPNmix
cell               [[MCF10A, IPN3], [MCF10A, IPN22], [MCF10A, IPN...
n_cells                                                            3
ignore                                                    [5, 7, 11]
stain                                                            NaN
dim                                                               3D
incubation_time                                                   72
seeding_density    [500, 500, 500, 500, 500, 500, 500, 500, 500, ...
passage_number                                                   NaN
other                                                          other
dt                                                            1800.0
n_fields                                                          15
n_frames                                                          45


Combining pkl, json, and csv 

In [135]:
#%%
#pixel_size = 0.32*1e-6
df_collecting = []
df_all = []

def search_csv(key_id):
    well_id = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'well_id'].iloc[0]
    measurement_id = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'measurement_id'].iloc[0]
    cell_label = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'cell_label'].iloc[0]
    matrix = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'matrix'].iloc[0]
    day = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'day'].iloc[0]
    spheroid_id = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'spheroid_id'].iloc[0]
    return well_id, measurement_id, cell_label, matrix, day, spheroid_id

def search_json(parameter):
    pixel_transf = Exp_2_json.get(key=str(parameter)).get("m")
    json_matrix = Exp_2_json.get(key=str(parameter)).get("matrix")
    json_cell = Exp_2_json.get(key=str(parameter)).get("cell")
    return pixel_transf,json_matrix,json_cell

df_collecting = []
df_all = []

for key in Exp_2_pckl.keys():
    df_temp = []

    if len(Exp_2_pckl[key]["time"]) == 0:
        print(f"No time data for key: {key}")
        continue  # Skips the next key

    for i in range(len(Exp_2_pckl[key]["time"])):
        mask = Exp_2_pckl[key]["masks"][i]
        time = Exp_2_pckl[key]["time"][i]
        
        well_id, measurement_id, cell_label, matrix, day, spheroid_id = search_csv(key)
        if any(val is None for val in [well_id, measurement_id, cell_label, matrix, day, spheroid_id]):
            print(f"Missing data for key: {key}")
            continue  # Skips to next key
        
        pixel_transf,json_matrix,json_cell = search_json(day)
        
        if pixel_transf is None:
            print(f"Missing pixel transformation data for date: {day}")
            continue  # Skip if no valid transformation is found
        
        area_pixels2 = cv2.contourArea(mask)
        area_µm = area_pixels2 * (pixel_transf ** 2)
        
        perimeter = cv2.arcLength(mask, closed=True) * pixel_transf
        compactness = 4 * math.pi * area_µm
        rect = cv2.minAreaRect(mask)
        width, height = rect[1]
        aspect_ratio = width / height
        convexhull = cv2.convexHull(mask)
        convexhull_perimeter = cv2.arcLength(convexhull, closed=True) * pixel_transf
        convexhull_area = cv2.contourArea(convexhull) * (pixel_transf ** 2)
        convexity_perimeter = convexhull_perimeter / perimeter
        convexity_area = convexhull_area / area_µm
        
        info = {
            "ID_running": key,
            "spheroid_id": spheroid_id,
            "time_id": i + 1,
            "incubation_time": time,
            "area_pixels**2": area_pixels2,
            "area_µm": area_µm,
            "perimeter_µm": perimeter,
            "compactness": compactness,
            "aspect_ratio": aspect_ratio,
            "convex hull perimeter": convexhull_perimeter,
            "convex hull area": convexhull_area,
            "convexity_perimeter": convexity_perimeter,
            "convexity_area": convexity_area,
            "well_id": well_id,
            "measurement_id": measurement_id,
            "cell_label": cell_label,
            "matrix": matrix,
            "json_matrix": json_matrix,
            "json_cell": json_cell,
            "day": day
        }
        df_temp.append(pd.DataFrame([info]))

    if df_temp:
        df_collecting.append(pd.concat(df_temp, ignore_index=True))
    else:
        print(f"No data to append for key: {key}")

df_all = pd.concat(df_collecting, ignore_index=True)


No time data for key: 323


In [136]:
print(len(df_all['spheroid_id'].unique()))
print(len(df_all['ID_running'].unique()))
print(len(df_all['well_id'].unique()))
print(len(df_all['measurement_id'].unique()))

182
407
22
19


In [137]:
df_normalized = df_all.copy()

Additional parameters into metadata

In [138]:

# Growth column ((size at t-size at zero)/size at zero)
for i in df_normalized['spheroid_id'].unique():  # looping through spheroids     
    specific_spheroid = df_normalized[df_normalized['spheroid_id'] == i].copy()
    min_time = np.argmin(specific_spheroid['incubation_time'])

    area_µm0 = specific_spheroid.iloc[min_time]['area_µm']
    compactness0 = specific_spheroid.iloc[min_time]['compactness']
    aspect_ratio0 = specific_spheroid.iloc[min_time]['aspect_ratio']
    convexity_perimeter0 = specific_spheroid.iloc[min_time]['convexity_perimeter']
    convexity_area0 = specific_spheroid.iloc[min_time]['convexity_area']
    
    # Compute the new columns
    specific_spheroid['growth_rate_%'] = ((specific_spheroid['area_µm'] - area_µm0) / area_µm0) * 100
    specific_spheroid['area_µm_N'] = specific_spheroid['area_µm'] / area_µm0
    specific_spheroid['compactness_N'] = specific_spheroid['compactness'] / compactness0
    specific_spheroid['aspect_ratio_N'] = specific_spheroid['aspect_ratio'] / aspect_ratio0
    specific_spheroid['convexity_perimeter_N'] = specific_spheroid['convexity_perimeter'] / convexity_perimeter0
    specific_spheroid['convexity_area_N'] = specific_spheroid['convexity_area'] / convexity_area0

    # Now, directly update the new columns in df_normalized
    df_normalized.loc[df_normalized['spheroid_id'] == i, 'growth_rate_%'] = specific_spheroid['growth_rate_%']
    df_normalized.loc[df_normalized['spheroid_id'] == i, 'area_µm_N'] = specific_spheroid['area_µm_N']
    df_normalized.loc[df_normalized['spheroid_id'] == i, 'compactness_N'] = specific_spheroid['compactness_N']
    df_normalized.loc[df_normalized['spheroid_id'] == i, 'aspect_ratio_N'] = specific_spheroid['aspect_ratio_N']
    df_normalized.loc[df_normalized['spheroid_id'] == i, 'convexity_perimeter_N'] = specific_spheroid['convexity_perimeter_N']
    df_normalized.loc[df_normalized['spheroid_id'] == i, 'convexity_area_N'] = specific_spheroid['convexity_area_N']

# Check the columns to ensure new ones are added
print(df_normalized.columns)

# Save the DataFrame to CSV
saving_path = 'C:/Users/srboval1/OneDrive - Aalto University/Instru/Datafiles/Exp2'
df_normalized.to_csv(saving_path + '/Exp2_data.csv')



Index(['ID_running', 'spheroid_id', 'time_id', 'incubation_time',
       'area_pixels**2', 'area_µm', 'perimeter_µm', 'compactness',
       'aspect_ratio', 'convex hull perimeter', 'convex hull area',
       'convexity_perimeter', 'convexity_area', 'well_id', 'measurement_id',
       'cell_label', 'matrix', 'json_matrix', 'json_cell', 'day',
       'growth_rate_%', 'area_µm_N', 'compactness_N', 'aspect_ratio_N',
       'convexity_perimeter_N', 'convexity_area_N'],
      dtype='object')


In [139]:
print(df_all.columns)


Index(['ID_running', 'spheroid_id', 'time_id', 'incubation_time',
       'area_pixels**2', 'area_µm', 'perimeter_µm', 'compactness',
       'aspect_ratio', 'convex hull perimeter', 'convex hull area',
       'convexity_perimeter', 'convexity_area', 'well_id', 'measurement_id',
       'cell_label', 'matrix', 'json_matrix', 'json_cell', 'day'],
      dtype='object')


Visualizing the mask

In [81]:
#%%
save = 'C:/Users/srboval1/OneDrive - Aalto University/Instru/Datafiles/TimeLapse/'
width, height = 2304, 2304

# Create a blank white image using Pillow and converting it into array


for key in Exp_2_pckl.keys() :
    time = Exp_2_pckl[key]["time"]
    
    blank_image = Image.new('RGB', (width, height), color = (255, 255, 255))
    blank_image_array = np.asarray(blank_image)

    new_image=np.zeros((height, width, 3), np.uint8)


    for i in range(len(Exp_2_pckl[key]["time"])): #looping through time
        mask = Exp_2_pckl[key]["masks"][i]
        spheroid_id = key+1
        blank_image_array= cv2.drawContours(blank_image_array, Exp_2_pckl[key]["masks"][i], -1, (0, 0, 0), 1)
        #print( f'{unique_material}- {unique_line}; min: {min_value} and max {max_value}')

    plt.title(f'{matrix} - {cell_line}; spheroid id: {spheroid_id}') 
    plt.imshow(new_image)
    filename = f'{cell_line}_{matrix}_id_{id}.tiff'

    cv2.imwrite(save+filename, blank_image_array) #saving the contours

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'drawContours'
> Overload resolution failed:
>  - image marked as output argument, but provided NumPy array marked as readonly
>  - Expected Ptr<cv::UMat> for argument 'image'
